In [2]:
!pip install sentence-transformers
!pip install faiss-cpu
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 73.9 MB/s eta 0:00:00


In [5]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [6]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# تحميل الملف
loader = TextLoader("/content/info.md", encoding="utf-8")
documents = loader.load()

# تقسيم النصوص إلى أجزاء صغيرة لتسهيل البحث
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("OmarAlsaabi/e5-base-mlqa-finetuned-arabic-for-rag")

embeddings = [model.encode(doc.page_content) for doc in docs]
embeddings = np.array(embeddings)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

In [8]:
import faiss

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)  # إضافة جميع Embeddings للـ index

In [9]:
query = "ما هي خدمات شركة الشفاء؟"
query_emb = model.encode([query])

# البحث عن النص الأقرب
D, I = index.search(query_emb, k=1)
print("أفضل جزء من الملف:", docs[I[0][0]].page_content)

أفضل جزء من الملف: شركة الشفاء الرقمية هي شركة رائدة ومتطورة في مجال **الرعاية الصحية عن بُعد (Telehealth)**، تأسست على مبدأ توفير رعاية طبية شاملة وعالية الجودة عبر الإنترنت. نهدف إلى سد الفجوة بين المرضى والأطباء المتخصصين، وتمكين الأفراد من الحصول على استشارات طبية موثوقة، متابعة مستمرة، وخدمات صحية متكاملة في أي وقت ومن أي مكان، وذلك بفضل أحدث التقنيات الرقمية والكوادر الطبية المتميزة. نحن نؤمن بأن الرعاية الصحية يجب أن تكون في متناول الجميع، ولذلك نعمل بلا كلل لتقديم حلول مبتكرة تسهل وصول المرضى للخدمات التي


In [10]:
# 1️⃣ البحث عن أفضل أجزاء من الملف
query = "ما هي خدمات شركة الشفاء؟"
query_emb = model.encode([query])
D, I = index.search(query_emb, k=3)  # نجيب أفضل 3 أجزاء
retrieved_texts = " ".join([docs[i].page_content for i in I[0]])

# 2️⃣ توليد إجابة كاملة باستخدام نموذج مولد
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# مثال: تستخدم أي نموذج لغة عربي مولد (يمكن تغييره لنموذج أفضل)
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-base")
model_gen = AutoModelForCausalLM.from_pretrained("aubmindlab/aragpt2-base")

prompt = f"استنادًا إلى المعلومات التالية:\n{retrieved_texts}\n\nجاوب على السؤال: {query}"

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model_gen.generate(**inputs, max_length=200)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("الإجابة الكاملة:", answer)

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


الإجابة الكاملة: استنادًا إلى المعلومات التالية:
شركة الشفاء الرقمية هي شركة رائدة ومتطورة في مجال **الرعاية الصحية عن بُعد (Telehealth)**، تأسست على مبدأ توفير رعاية طبية شاملة وعالية الجودة عبر الإنترنت. نهدف إلى سد الفجوة بين المرضى والأطباء المتخصصين، وتمكين الأفراد من الحصول على استشارات طبية موثوقة، متابعة مستمرة، وخدمات صحية متكاملة في أي وقت ومن أي مكان، وذلك بفضل أحدث التقنيات الرقمية والكوادر الطبية المتميزة. نحن نؤمن بأن الرعاية الصحية يجب أن تكون في متناول الجميع، ولذلك نعمل بلا كلل لتقديم حلول مبتكرة تسهل وصول المرضى للخدمات التي # شركة الشفاء الرقمية للرعاية الصحية

## 🌟 نبذة عن الشركة ---

## 🩺 الخدمات التي نقدمها

نقدم في شركة الشفاء الرقمية مجموعة واسعة ومتكاملة من الخدمات الصحية الرقمية التي تلبي مختلف احتياجات الرعاية:

جاوب على السؤال: ما هي خدمات شركة الشفاء؟�غ الخدمات التي نقدمها لنقدمها في شركة


In [11]:
# -------------------------------
# 1️⃣ البحث عن أفضل أجزاء من الملف
query = "إذا كنت مشترك في باقة سنوية وقررت ألغي الخدمة، هل أقدر أسترد المبلغ بالكامل؟ ولو نعم، خلال قد إيه؟"
query_emb = model.encode([query])
D, I = index.search(query_emb, k=3)  # نجيب أفضل 3 أجزاء
retrieved_texts = " ".join([docs[i].page_content for i in I[0]])

# -------------------------------
# 2️⃣ توليد إجابة كاملة باستخدام نموذج مولد
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# نموذج عربي مولد (يمكن تغييره لنموذج أفضل)
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-base")
model_gen = AutoModelForCausalLM.from_pretrained("aubmindlab/aragpt2-base")

prompt = f"استنادًا إلى المعلومات التالية:\n{retrieved_texts}\n\nجاوب على السؤال التالي بطريقة واضحة ومفصلة:\n{query}"

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model_gen.generate(
    **inputs,
    max_length=400,              # زيادة طول النص لتفادي القطع
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,              # لتوليد إجابة أكثر طبيعية
    top_p=0.9,
    temperature=0.7
)

answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

# -------------------------------
# 3️⃣ طباعة الإجابة بطريقة أفضل
print("\n" + "="*50)
print("💬 السؤال:")
print(query)
print("\n📝 الإجابة:")
print(answer)
print("="*50 + "\n")


💬 السؤال:
إذا كنت مشترك في باقة سنوية وقررت ألغي الخدمة، هل أقدر أسترد المبلغ بالكامل؟ ولو نعم، خلال قد إيه؟

📝 الإجابة:
استنادًا إلى المعلومات التالية:
**س: هل يمكنني استرداد المبلغ إذا لم أكن راضيًا عن الخدمة؟**
ج: نعم، نسعى دائمًا لرضاكم التام. يمكن طلب استرداد المبلغ خلال **24 ساعة** من تقديم الخدمة أو عدم إتمامها، وذلك وفقًا لسياسة الاسترداد الخاصة بنا، والتي تحدد الشروط والأحكام لكل خدمة. يرجى مراجعة صفحة "شروط الاستخدام" للحصول على تفاصيل كاملة. **ملاحظة:** يرجى مراجعة شروط الاشتراك في كل باقة، حيث قد تختلف سياسات الإلغاء والاسترداد بناءً على نوع الخدمة أو مدة الاشتراك. * **عبر حسابك:** قم بتسجيل الدخول إلى حسابك على موقعنا الإلكتروني. ثم توجه إلى "إعدادات الحساب" أو "إدارة الاشتراكات" (قد تختلف التسمية قليلاً حسب التحديثات). اختر "إلغاء الاشتراك" واتبع التعليمات. سيتم تفعيل الإلغاء فورًا أو في نهاية فترة الاشتراك الحالية حسب نوع الباقة.

جاوب على السؤال التالي بطريقة واضحة ومفصلة:
إذا كنت مشترك في باقة سنوية وقررت ألغي الخدمة، هل أقدر أسترد المبلغ بالكامل؟ ولو نعم، خلال قد إيه

In [12]:
# -------------------------------
# 1️⃣ البحث عن أفضل أجزاء من الملف
query = "إذا كنت مشترك في باقة سنوية وقررت ألغي الخدمة، هل أقدر أسترد المبلغ بالكامل؟ ولو نعم، خلال قد إيه؟"
query_emb = model.encode([query])
D, I = index.search(query_emb, k=3)
retrieved_texts = " ".join([docs[i].page_content for i in I[0]])

# -------------------------------
# 2️⃣ توليد إجابة كاملة باستخدام نموذج مولد
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-base")
model_gen = AutoModelForCausalLM.from_pretrained("aubmindlab/aragpt2-base")

prompt = f"استنادًا إلى المعلومات التالية: {retrieved_texts}\nجاوب على السؤال بأسلوب واضح ومفصل: {query}"

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model_gen.generate(
    **inputs,
    max_new_tokens=250,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    top_p=0.9,
    temperature=0.7
)

answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

# إزالة أي تكرار للسؤال في نهاية النص
if query in answer:
    answer = answer.split(query)[0].strip()

# -------------------------------
# 3️⃣ طباعة النص النهائي بشكل نظيف للتصدير لـ TTS
print("\n" + "-"*50)
print("الإجابة على سؤالك:")
print(answer)
print("-"*50 + "\n")


--------------------------------------------------
الإجابة على سؤالك:
استنادًا إلى المعلومات التالية: **س: هل يمكنني استرداد المبلغ إذا لم أكن راضيًا عن الخدمة؟**
ج: نعم، نسعى دائمًا لرضاكم التام. يمكن طلب استرداد المبلغ خلال **24 ساعة** من تقديم الخدمة أو عدم إتمامها، وذلك وفقًا لسياسة الاسترداد الخاصة بنا، والتي تحدد الشروط والأحكام لكل خدمة. يرجى مراجعة صفحة "شروط الاستخدام" للحصول على تفاصيل كاملة. **ملاحظة:** يرجى مراجعة شروط الاشتراك في كل باقة، حيث قد تختلف سياسات الإلغاء والاسترداد بناءً على نوع الخدمة أو مدة الاشتراك. * **عبر حسابك:** قم بتسجيل الدخول إلى حسابك على موقعنا الإلكتروني. ثم توجه إلى "إعدادات الحساب" أو "إدارة الاشتراكات" (قد تختلف التسمية قليلاً حسب التحديثات). اختر "إلغاء الاشتراك" واتبع التعليمات. سيتم تفعيل الإلغاء فورًا أو في نهاية فترة الاشتراك الحالية حسب نوع الباقة.
جاوب على السؤال بأسلوب واضح ومفصل:
--------------------------------------------------



In [13]:
# -------------------------------
# 1️⃣ البحث عن أفضل أجزاء من الملف
query = "إذا كنت مشترك في باقة سنوية وقررت ألغي الخدمة، هل أقدر أسترد المبلغ بالكامل؟ ولو نعم، خلال قد إيه؟"
query_emb = model.encode([query])
D, I = index.search(query_emb, k=3)
retrieved_texts = " ".join([docs[i].page_content for i in I[0]])

# -------------------------------
# 2️⃣ توليد إجابة كاملة بدون تكرار السؤال
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-base")
model_gen = AutoModelForCausalLM.from_pretrained("aubmindlab/aragpt2-base")

# **هنا نعدل الـ prompt**: نطلب توليد الإجابة فقط
prompt = f"استنادًا إلى المعلومات التالية: {retrieved_texts}\nقدم إجابة واضحة ومفصلة على السؤال التالي:\n{query}\n\nالإجابة:"

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model_gen.generate(
    **inputs,
    max_new_tokens=250,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    eos_token_id=tokenizer.eos_token_id  # يضمن انتهاء الإجابة عند نهاية النص
)

answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

# -------------------------------
# 3️⃣ تنظيف الإجابة بشكل نهائي للتصدير لـ TTS
# إزالة أي ظهور للسؤال في الإجابة
if query in answer:
    answer = answer.split(query)[0].strip()

# إزالة أي نص إضافي بعد "الإجابة:" إذا ظهر
if "الإجابة:" in answer:
    answer = answer.split("الإجابة:")[-1].strip()

# -------------------------------
# 4️⃣ الطباعة النهائية
print("\n" + "-"*50)
print("الإجابة على سؤالك:")
print(answer)
print("-"*50 + "\n")


--------------------------------------------------
الإجابة على سؤالك:
استنادًا إلى المعلومات التالية: **س: هل يمكنني استرداد المبلغ إذا لم أكن راضيًا عن الخدمة؟**
ج: نعم، نسعى دائمًا لرضاكم التام. يمكن طلب استرداد المبلغ خلال **24 ساعة** من تقديم الخدمة أو عدم إتمامها، وذلك وفقًا لسياسة الاسترداد الخاصة بنا، والتي تحدد الشروط والأحكام لكل خدمة. يرجى مراجعة صفحة "شروط الاستخدام" للحصول على تفاصيل كاملة. **ملاحظة:** يرجى مراجعة شروط الاشتراك في كل باقة، حيث قد تختلف سياسات الإلغاء والاسترداد بناءً على نوع الخدمة أو مدة الاشتراك. * **عبر حسابك:** قم بتسجيل الدخول إلى حسابك على موقعنا الإلكتروني. ثم توجه إلى "إعدادات الحساب" أو "إدارة الاشتراكات" (قد تختلف التسمية قليلاً حسب التحديثات). اختر "إلغاء الاشتراك" واتبع التعليمات. سيتم تفعيل الإلغاء فورًا أو في نهاية فترة الاشتراك الحالية حسب نوع الباقة.
قدم إجابة واضحة ومفصلة على السؤال التالي:
--------------------------------------------------

